In [1]:
# Setup the environment
!pip install -q -U immutabledict sentencepiece 
!git clone https://github.com/google/gemma_pytorch.git
!mkdir /kaggle/working/gemma/
!mv /kaggle/working/gemma_pytorch/gemma/* /kaggle/working/gemma/



Cloning into 'gemma_pytorch'...
remote: Enumerating objects: 133, done.
remote: Counting objects: 100% (65/65), done.
remote: Compressing objects: 100% (41/41), done.
remote: Total 133 (delta 40), reused 35 (delta 23), pack-reused 68
Receiving objects: 100% (133/133), 2.15 MiB | 6.18 MiB/s, done.
Resolving deltas: 100% (67/67), done.


In [2]:
import sys 
sys.path.append("/kaggle/working/gemma_pytorch/") 
from gemma.config import GemmaConfig, get_config_for_7b, get_config_for_2b
from gemma.model import GemmaForCausalLM
from gemma.tokenizer import Tokenizer
import contextlib
import os
import torch
import json
import random
import pandas as pd

# Load the model
VARIANT = "7b-it-quant" 
MACHINE_TYPE = "cuda" 
weights_dir = '/kaggle/input/gemma/pytorch/7b-it-quant/2' 

@contextlib.contextmanager
def _set_default_tensor_type(dtype: torch.dtype):
  """Sets the default torch dtype to the given dtype."""
  torch.set_default_dtype(dtype)
  yield
  torch.set_default_dtype(torch.float)

model_config = get_config_for_2b() if "2b" in VARIANT else get_config_for_7b()
model_config.tokenizer = os.path.join(weights_dir, "tokenizer.model")
model_config.quant = "quant" in VARIANT

device = torch.device(MACHINE_TYPE)
with _set_default_tensor_type(model_config.get_dtype()):
  model = GemmaForCausalLM(model_config)
  ckpt_path = os.path.join(weights_dir, f'gemma-{VARIANT}.ckpt')
  model.load_weights(ckpt_path)
  model = model.to(device).eval()



# Use the model

USER_CHAT_TEMPLATE = "<start_of_turn>user\n{prompt}<end_of_turn>\n"
MODEL_CHAT_TEMPLATE = "<start_of_turn>model\n{prompt}<end_of_turn>\n"

prompt = (
    USER_CHAT_TEMPLATE.format(
        prompt="What is a good place for travel in the US?"
    )
    + MODEL_CHAT_TEMPLATE.format(prompt="California.")
    + USER_CHAT_TEMPLATE.format(prompt="What can I do in California?")
    + "<start_of_turn>model\n"
)

model.generate(
    USER_CHAT_TEMPLATE.format(prompt=prompt),
    device=device,
    output_len=100,
)


/opt/conda/lib/python3.10/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


"There's a lot to do in California! Here's a quick breakdown of some of the most popular activities:\n\n**For nature lovers:**\n* Visit redwood trees in Northern California's Redwood National Park or Sequoia National Park.\n* Hike to the top of Mount Shasta or Yosemite Falls.\n* Drive along the scenic Highway 1 along the coast of Northern California.\n* Explore the Golden Gate Bridge and Golden Gate Park in San Francisco.\n* Take a boat trip to Al"

In [3]:
class PromptGenerator(object):
    def __init__(self):
        """
        Setup generation parameters for Gemma.
        """
        self.output_len = 20
        self.temperature = 0.0
        self.top_p = 1.0

    def create_prompt(self, sample):
        """
        Input sample is a dictionary consisting of following fields
        'table': A dict containing table data and meta-data same as in Assignment 2
        'question': A python string for the question on the table.
        
        The function must return the prompt as a python string.
        """
        # Example prompt
        prompt = """Read the following table and answer the related question.\n\n"""
        prompt += 'TABLE:\n'
        prompt += ','.join(
            [f'"{cc}"' for cc in sample['table']['cols']]
        ) + '\n'
        for row in sample['table']['rows']:
            prompt += ','.join(
                [f'"{rr}"' for rr in row]
            ) + '\n'
        prompt += '\n'
        prompt += 'QUESTION: ' + sample['question'] + '\n'
        prompt += 'Now give the correct column '

        return prompt
    
    def create_prompt_shot(self, sample):
        """
        Input sample is a dictionary consisting of following fields
        'table': A dict containing table data and meta-data same as in Assignment 2
        'question': A python string for the question on the table.
        
        The function must return the prompt as a python string.
        """
        # Example prompt
        prompt = """Read the following table and answer the related question.\n\n"""
        prompt += 'TABLE:\n'
        prompt += ','.join(
            [f'"{cc}"' for cc in sample['table']['cols']]
        ) + '\n'
        for row in sample['table']['rows']:
            prompt += ','.join(
                [f'"{rr}"' for rr in row]
            ) + '\n'
        prompt += '\n'
        prompt += 'QUESTION: ' + sample['question'] + '\n'
#         print(sample['label_col'])
        prompt += 'The correct column is ' + sample['label_col'] + '\n'
        prompt += '\n'

        return prompt
    
    def post_process(self, gen_text):
        """
        Input gen_text is a python string generated by Gemma for the prompt.
        
        The function must return a single python tuple (int, string)
        indicating the row and the column of the answer cell.
        """
        gen_text = gen_text.split('\n')[0]
        # now remove the first 4 words from the generated text
        gen_text = ' '.join(gen_text.split()[4:])
        return gen_text

    

In [4]:
sample_shot = {
    "question":"When was the opponent Poland and the match type EC -qualifier?",
    "table":{
        "cols":["Date","Location","Opponenent","Result","Match type"],
        "rows":[
            ["29 March 2000","Debrecen","Poland","0-0 (draw)","friendly"],
            ["16 August 2000","Budapest","Austria","1-1 (draw)","friendly"],
            ["3 September 2000","Budapest","Italy","2-2 (draw)","WC -qualifier"],
            ["15 August 2001","Budapest","Germany","2-5 (defeat)","friendly"],
            ["1 September 2001","Tbilisi","Georgia","1-3 (defeat)","WC-qualifier"],
            ["5 September 2001","Budapest","Romania","0-2 (defeat)","WC-qualifier"],
            ["6 October 2001","Parma","Italy","0-0 (draw)","WC-qualifier"],
            ["14 November 2001","Budapest","Macedonia","5-0 (win)","friendly"],
            ["12 February 2002","Larnaca","Czech Rep.","0-2 (defeat)","friendly"],
            ["13 February 2002","Limassol","Switzerland","1-2 (defeat)","friendly"],
            ["8 May 2002","P\u00e9cs","Croatia","0-2 (defeat)","friendly"],
            ["21 August 2002","Budapest","Spain","1-1 (draw)","friendly"],
            ["7 September 2002","Reykjav\u00edk","Iceland","2-0 (win)","friendly"],
            ["12 October 2002","Stockholm","Sweden","1-1 (draw)","EC-qualifier"],
            ["16 October 2002","Budapest","San Marino","3-0 (win)","EC -qualifier"],
            ["20 November 2002","Budapest","Moldova","1-1 (draw)","friendly"],
            ["12 February 2003","Larnaca","Bulgaria","0-1 (defeat)","friendly"],
            ["29 March 2003","Chorz\u00f3w","Poland","0-0 (draw)","EC -qualifier"],
            ["2 April 2003","Budapest","Sweden","1-2 (defeat)","EC -qualifier"],
            ["30 April 2003","Budapest","Luxembourg","5-1 (win)","friendly"],
            ["19 February 2004","Limassol","Latvia","2-1 (win)","friendly"],
            ["21 February 2004","Limassol","Romania","0-3 (defeat)","friendly"],
            ["9 February 2011","Dubai","Azerbaijan","2-0 (win)","friendly"],
            ["29 March 2011","Amsterdam","Netherlands","3-5 (defeat)","EC -qualifier"],
            ["3 June 2011","Luxembourg","Luxembourg","1-0 (win)","friendly"]
        ],
        "types":["text","text","text","text","text"],
        "caption":"National team matches",
    },
    "label_col":"Date"
}
sample_shot

sample_shot2 = {
    "question":"What is the total score when 7 is the average ranking?",
    "table":{
        "cols":["Average Ranking","Competitive Finish","Couple","Number Of Dances","Total Score","Average"],
        "rows":[
            ["1","1","Bridie & Craig","15","509","35.9"],
            ["2","3","David & Karina","12","360","30.0"],
            ["3","4","Patti & Sandro","10","295","29.5"],
            ["4","2","Anh & Luda","15","421","27.0"],
            ["5","9","Corinne & Csaba","3","77","25.7"],
            ["6","5","Mark & Linda","8","204","25.5"],
            ["7","8","Elka & Michael","4","100","25.0"],
            ["8","6","James & Olya","7","169","24.1"],
            ["9","7","Jessica & Serghei","5","120","24.0"]
        ],
        "types":["real","real","text","real","real","text"],
        "caption":"Average Chart"
    },
    "label_col":"Total Score"
}
sample_shot2

{'question': 'What is the total score when 7 is the average ranking?',
 'table': {'cols': ['Average Ranking',
   'Competitive Finish',
   'Couple',
   'Number Of Dances',
   'Total Score',
   'Average'],
  'rows': [['1', '1', 'Bridie & Craig', '15', '509', '35.9'],
   ['2', '3', 'David & Karina', '12', '360', '30.0'],
   ['3', '4', 'Patti & Sandro', '10', '295', '29.5'],
   ['4', '2', 'Anh & Luda', '15', '421', '27.0'],
   ['5', '9', 'Corinne & Csaba', '3', '77', '25.7'],
   ['6', '5', 'Mark & Linda', '8', '204', '25.5'],
   ['7', '8', 'Elka & Michael', '4', '100', '25.0'],
   ['8', '6', 'James & Olya', '7', '169', '24.1'],
   ['9', '7', 'Jessica & Serghei', '5', '120', '24.0']],
  'types': ['real', 'real', 'text', 'real', 'real', 'text'],
  'caption': 'Average Chart'},
 'label_col': 'Total Score'}

In [5]:
# sample = {
#     "question":"What is the sum of week(s) with an attendance of 30,751?",
#     "table":{
#         "cols":["Week","Date","Opponent","Result","Attendance"],
#         "rows":[
#             ["1","August 6, 1973","San Francisco 49ers","L 27\u201316","65,707"],
#             ["2","August 11, 1973","at Los Angeles Rams","T 21\u201321","54,385"],
#             ["3","August 19, 1973","vs. Cincinnati Bengals at Columbus, Ohio","W 24\u20136","73,421"],
#             ["4","August 25, 1973","vs. Atlanta Falcons at Knoxville","W 20\u201317","40,831"],
#             ["5","September 1, 1973","Detroit Lions","L 16\u201313","64,088"],
#             ["6","September 8, 1973","vs. New York Giants at Akron","L 21\u201310","30,751"]
#         ],
#         "types":["real","text","text","text","real"],
#         "caption":"Exhibition schedule"
#     }
# }
# sample
# prompt_gen = PromptGenerator()
# prompt_act = prompt_gen.create_prompt(sample)
# prompt_shot = prompt_gen.create_prompt_shot(sample_shot)
# prompt_shot2 = prompt_gen.create_prompt_shot(sample_shot2)
# prompt = prompt_shot + prompt_shot2 + prompt_act
# print(prompt)

# gen_text = model.generate(
#     USER_CHAT_TEMPLATE.format(prompt=prompt),
#     device=device,
#     output_len=prompt_gen.output_len,
#     temperature=prompt_gen.temperature,
#     top_p=prompt_gen.top_p,
# )

# print(gen_text)

# answer = prompt_gen.post_process(gen_text)
# answer

In [6]:
data = []
with open('/kaggle/input/a2-val/A2_val.jsonl', 'r') as f:
    for line in f:
        data.append(json.loads(line))

# Sample 100 random samples
random.seed(42)
samples = random.sample(data, 300)



In [7]:
# Generate the answers
prompt_generator = PromptGenerator()
prompt_shot = prompt_generator.create_prompt_shot(sample_shot)
prompt_shot2 = prompt_generator.create_prompt_shot(sample_shot2)
answers = []
num=1
maxrow=0
for sample in samples:
    print("Sample number is "+ str(num))
    print("Question is "+ sample['question'])
    ## print the number of rows and columns in the table
#     print("Number of rows in the table is "+ str(len(sample['table']['rows'])))
#     print("Number of columns in the table is "+ str(len(sample['table']['cols'])))
    if len(sample['table']['rows']) > 100:
        sample['table']['rows'] = sample['table']['rows'][:100]
    cur_sample = {'question': sample['question'], 'table': sample['table']}
    maxrow = max(maxrow,len(sample['table']['rows']))
    prompt_act = prompt_generator.create_prompt(cur_sample)
    prompt = prompt_shot + prompt_shot2 + prompt_act
    answer = model.generate(
        USER_CHAT_TEMPLATE.format(prompt=prompt),
        device=device,
        output_len=prompt_generator.output_len,
        temperature=prompt_generator.temperature,
        top_p=prompt_generator.top_p,
    )
#     print("Gen text is "+ answer)
    answer = prompt_generator.post_process(answer)
    print("Answer got is " + answer)
    print("Answer should be "+ sample['label_col'][0])
    print("===================================================================")
    answers.append(answer)
    num+=1
    del cur_sample
    del prompt_act
    del prompt
    del answer

# Now find the accuracy of the model
correct = 0
for sample, answer in zip(samples, answers):
    if sample['label_col'][0] == answer:
        correct += 1

accuracy = correct / len(samples)
print("accuracy is "+ accuracy)
print("maxrow is "+ maxrow)
print("correct ans is "+ correct)

Sample number is 1
Question is Who is Winners that has  editions of 1992, 2003 as Not Winning
Answer got is Winners
Answer should be Winners
Sample number is 2
Question is What was the name for the row with Date From of 2008-02-21?
Answer got is Name
Answer should be Name
Sample number is 3
Question is What's listed as the First Day Cover Cancellation with a Date of Issue of 13 June 2005?
Answer got is First Day Cover Cancellation
Answer should be First Day Cover Cancellation
Sample number is 4
Question is what is the nme of the song performed by billy vaughn?
Answer got is Song title
Answer should be Song title
Sample number is 5
Question is Which Driver has a Sponsor of pdvsa?
Answer got is Test Drivers
Answer should be Driver(s)
Sample number is 6
Question is What home team played against St Kilda?
Answer got is Home team
Answer should be Home team
Sample number is 7
Question is I want the highest Grid for Toyota and jarno trulli
Answer got is Grid
Answer should be Grid
Sample numbe

OutOfMemoryError: CUDA out of memory. Tried to allocate 2.03 GiB. GPU 0 has a total capacty of 15.89 GiB of which 2.04 GiB is free. Process 2566 has 13.86 GiB memory in use. Of the allocated memory 13.05 GiB is allocated by PyTorch, and 530.92 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [ ]:
print(correct)